#1. Dados de batimetria
##1.1 Os dados de batimetria foram baixados de https://maps.ngdc.noaa.gov/viewers/grid-extract/index.html
##1.2 O output desse arquivo deve ser os dados de batimetria com (lat,lon) em uma resolução de 1km

##1.3 Importar as bibliotecas

In [3]:
from osgeo import gdal
import xarray as xr
import numpy as np


##1.4 Os dados foram dispostos em um formato raster, vou traduzir eles para netcdf4 por ser mais simples de trabalhar com a biblioteca xarray

In [3]:
inputfile = '/content/drive/MyDrive/exportImage.tiff'
outputfile = '/content/drive/MyDrive/exportImage1.nc'
ds = gdal.Translate(outputfile, inputfile, format='NetCDF')

##1.5 com os dados em formato nc lê-se eles com a biblioteca xarray

In [4]:
x = xr.open_dataset('/content/drive/MyDrive/exportImage1.nc')

In [5]:
x

<xarray.Dataset>
Dimensions:  (lat: 3600, lon: 5400)
Coordinates:
  * lat      (lat) float64 -20.0 -19.98 -19.97 -19.95 ... 39.95 39.97 39.98
  * lon      (lon) float64 -80.0 -79.98 -79.97 -79.95 ... 9.933 9.95 9.967 9.983
Data variables:
    crs      |S1 ...
    Band1    (lat, lon) float32 ...
Attributes:
    GDAL_AREA_OR_POINT:  Area
    GDAL_DataType:       Generic
    Conventions:         CF-1.5
    GDAL:                GDAL 2.2.3, released 2017/11/20
    history:             Thu Nov 18 20:49:39 2021: GDAL CreateCopy( /content/...

##1.6 Agora é montar um dataframe pandas para os valores

###1.6.1 recuperar os dados disponiveis

In [6]:
latitude = x.get(key='lat')
longitude = x.get(key='lon')
batimetriaVetor = x.get(key='Band1')

###1.6.2 setar as variaveis e o dataframe

In [20]:
import pandas as pd
df = list()

###1.6.3 montar o array
  > como os dados estão na resolução de 1 metro, vou por todos eles em resolução de 1 km 

In [21]:
for lat in range(len(latitude)):
  for lon in range(len(longitude)):
    la = round(latitude.data[lat],2)
    lo = round(longitude.data[lon],2)
    ba = batimetriaVetor.data[lat][lon]
    linha = [la,lo,ba]
    df.append(linha)

###1.6.4 montar o dataframe

In [23]:
df = pd.DataFrame(data=df,columns=['Latitude','Longitude','Batimetria'])

##1.7 Com os dados com resolução igual a do satelite de temperatura, não foi preciso agrupar porque os dados estavam bem separados, com isso o dataFrame vai ser salvo

In [ ]:
df.to_parquet('/content/drive/MyDrive/batimetria')